In [ ]:
import pandas as pd

In [ ]:
#load data
a = pd.read_csv('~/datasets/s4/MENTORSHIP/authorship.csv')
pids_sample  = pd.read_csv('pids_sample_v1.csv')

In [ ]:
#get list of papers for which we need to calculate citation count
a = a.merge(pids_sample, on='PID', how='inner')
magids = a['MAGPaperID'].unique()

In [ ]:
#filter 'PaperReferences.txt' for citations to papers in sample

refs = []

ll = 0
chunksize = 1000
mag_ref_file = '~/datasets/s4/MAG/PaperReferences.txt'

for d in (pd.read_csv(mag_ref_file, sep='\t', header=None, chunksize=chunksize)):
    ll += 1
    print(f'\rprocessing {mag_ref_file} chunk {ll}', end='')
    d.columns = ['PaperId','PaperReferenceId']
    in_sample = d.loc[d['PaperReferenceId'].isin(magids)]
    refs.append(in_sample)

refs = pd.concat(refs)

In [ ]:
#filter 'Papers.txt' for papers that cite those those in sample, only keep publication year
refids = refs['PaperId'].unique()

# header_df = pd.read_csv('~/datasets/s4/MAG/Papers.csv')
# header = header_df.columns.values.tolist() + ['FamilyID', 'FamilyRank', 'CreatedDate']

ref_years = []

ll = 0
chunksize = 1000
mag_pub_file = '~/datasets/s4/MAG/Papers.txt'

for d in (pd.read_csv(mag_pub_file, sep='\t', header=None, usecols=[0,7], chunksize=chunksize)):
    ll += 1
    print(f'\rprocessing {mag_pub_file} chunk {ll}', end='')
    d.columns = ['PaperId','Year']
    in_sample = d.loc[d['PaperId'].isin(refids)]
    ref_years.append(in_sample)

In [ ]:
#merge data
d = pd.merge(refs, ref_years, left_on='PaperReferenceId', right_on='PaperId')

In [ ]:
d.to_hdf('citation_v1.hd5', key='citation', complevel=9)